In [ ]:
# Importing necessary libraries
import nltk
nltk.download('punkt') # Download the Punkt tokenizer models
nltk.download('wordnet') # Download the WordNet lexical database
from nltk.stem import WordNetLemmatizer # Import the WordNet lemmatizer
lemmatizer = WordNetLemmatizer() # Initialize the lemmatizer
import pickle # Import pickle for serializing and deserializing Python object structures
import numpy as np # Import numpy for array operations

from keras.models import load_model # Import load_model from Keras to load the trained model
model = load_model('model.h5') # Load the trained model from file
import json # Import json for handling JSON data
import random # Import random for random selections

# Load intents and model data
intents = json.loads(open('data.json').read()) # Load the intents JSON file
words = pickle.load(open('texts.pkl', 'rb')) # Load the words used in training (vocabulary)
classes = pickle.load(open('labels.pkl', 'rb')) # Load the class labels

# Function to clean up a sentence (tokenize and lemmatize)
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence) # Tokenize the sentence into words
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words] # Lemmatize each word
    return sentence_words

# Function to convert a sentence into a bag-of-words vector
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence) # Clean up the sentence
    bag = [0] * len(words) # Initialize the bag of words with 0s
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s: 
                bag[i] = 1 # If the word is in the sentence, mark it as 1 in the bag
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

# Function to predict the class of an input sentence
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False) # Convert sentence to bag-of-words
    res = model.predict(np.array([p]))[0] # Predict the class probabilities
    ERROR_THRESHOLD = 0.25 # Set a threshold to filter out weak predictions
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD] # Filter predictions above the threshold
    results.sort(key=lambda x: x[1], reverse=True) # Sort by probability in descending order
    return_list = [{"intent": classes[r[0]], "probability": str(r[1])} for r in results] # Create a list of intents and their probabilities
    return return_list

# Function to get a response based on predicted intent
def getResponse(ints, intents_json):
    tag = ints[0]['intent'] # Get the intent with the highest probability
    for intent in intents_json['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses']) # Return a random response for the matched intent
    return "Sorry, I didn't understand that."

# Function to generate a chatbot response
def chatbot_response(msg):
    ints = predict_class(msg, model) # Predict the intent of the message
    res = getResponse(ints, intents) # Get the appropriate response
    return res

# Flask app setup
from flask import Flask, render_template, request

app = Flask(__name__) # Initialize the Flask app
app.static_folder = 'static' # Set the static folder

@app.route("/")
def home():
    return render_template("index.html") # Render the home page

@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg') # Get the user message from the request
    return chatbot_response(userText) # Return the chatbot response

if __name__ == "__main__":
    app.run() # Run the Flask app


[nltk_data] Downloading package punkt to C:\Users\venkat
[nltk_data]     naidu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\venkat
[nltk_data]     naidu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
